In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pandas
import matplotlib
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))


spark_session = SparkSession\
        .builder\
        .master("spark://192.168.2.113:7077") \
        .appName("AdityaShirke_B")\
        .config("spark.dynamicAllocation.enabled", True)\
        .config("spark.shuffle.service.enabled", True)\
        .config("spark.dynamicAllocation.executorIdleTimeout","30s")\
        .config("spark.executor.cores",2)\
        .config("spark.driver.port",9998)\
        .config("spark.blockManager.port",10005)\
        .getOrCreate()

In [13]:
#B.1 Load the CSV file from HDFS, and call show() to verify the data is loaded correctly.
citations = spark_session.read.csv("hdfs://192.168.2.113:9000/parking-citations.csv", header = True, sep = ",")
citations.take(10)

[Row(Ticket number='1103341116', Issue Date='2015-12-21T00:00:00', Issue time='1251', Meter Id=None, Marked Time=None, RP State Plate='CA', Plate Expiry Date='200304', VIN=None, Make='HOND', Body Style='PA', Color='GY', Location='13147 WELBY WAY', Route='01521', Agency='1', Violation code='4000A1', Violation Description='NO EVIDENCE OF REG', Fine amount='50', Latitude='99999', Longitude='99999'),
 Row(Ticket number='1103700150', Issue Date='2015-12-21T00:00:00', Issue time='1435', Meter Id=None, Marked Time=None, RP State Plate='CA', Plate Expiry Date='201512', VIN=None, Make='GMC', Body Style='VN', Color='WH', Location='525 S MAIN ST', Route='1C51', Agency='1', Violation code='4000A1', Violation Description='NO EVIDENCE OF REG', Fine amount='50', Latitude='99999', Longitude='99999'),
 Row(Ticket number='1104803000', Issue Date='2015-12-21T00:00:00', Issue time='2055', Meter Id=None, Marked Time=None, RP State Plate='CA', Plate Expiry Date='201503', VIN=None, Make='NISS', Body Style='P

In [25]:
#B.2 Print the schema for the DataFrame.
citations.show()

+-------------+-------------------+----------+--------+-----------+--------------+-----------------+----+----+----------+-----+--------------------+-----+------+--------------+---------------------+-----------+---------+---------+
|Ticket number|         Issue Date|Issue time|Meter Id|Marked Time|RP State Plate|Plate Expiry Date| VIN|Make|Body Style|Color|            Location|Route|Agency|Violation code|Violation Description|Fine amount| Latitude|Longitude|
+-------------+-------------------+----------+--------+-----------+--------------+-----------------+----+----+----------+-----+--------------------+-----+------+--------------+---------------------+-----------+---------+---------+
|   1103341116|2015-12-21T00:00:00|      1251|    null|       null|            CA|           200304|null|HOND|        PA|   GY|     13147 WELBY WAY|01521|     1|        4000A1|   NO EVIDENCE OF REG|         50|    99999|    99999|
|   1103700150|2015-12-21T00:00:00|      1435|    null|       null|         

In [12]:
#B.3 Count the number of rows in the CSV file.
citations.count()

9257460

In [17]:
#B.4 Count the number of partitions in the underlying RDD.
citations.rdd.getNumPartitions()

10

In [26]:
#B.5 Drop the columns VIN, Latitude and Longitude.
citations_new = citations.drop("VIN", "Latitude", "Longitude")
citations_new.show()

#Another way
#citations_new = citations.drop(citations.VIN).drop(citations.Latitude).drop(citations.Longitude)
#citations_new.show()

+-------------+-------------------+----------+--------+-----------+--------------+-----------------+----+----------+-----+--------------------+-----+------+--------------+---------------------+-----------+
|Ticket number|         Issue Date|Issue time|Meter Id|Marked Time|RP State Plate|Plate Expiry Date|Make|Body Style|Color|            Location|Route|Agency|Violation code|Violation Description|Fine amount|
+-------------+-------------------+----------+--------+-----------+--------------+-----------------+----+----------+-----+--------------------+-----+------+--------------+---------------------+-----------+
|   1103341116|2015-12-21T00:00:00|      1251|    null|       null|            CA|           200304|HOND|        PA|   GY|     13147 WELBY WAY|01521|     1|        4000A1|   NO EVIDENCE OF REG|         50|
|   1103700150|2015-12-21T00:00:00|      1435|    null|       null|            CA|           201512| GMC|        VN|   WH|       525 S MAIN ST| 1C51|     1|        4000A1|   NO

In [108]:
#B.6 Find the maximum fine amount. How many fines have this amount?
#citations_new_floatAmount = citations_new.withColumn("Fine amount", citations_new['Fine amount'].cast('float'))

#citations_new_floatAmount.show()


row1 = citations_new_floatAmount.agg({"Fine amount": "max"}).collect()[0]
print(row1["max(Fine amount)"])


row2 = citations_new_floatAmount.agg({"Fine amount": "max"}).count()
print(row2)



from pyspark.sql.functions import col,when,count

#citations_new_floatAmount.groupBy("Fine amount").agg(
#    count(when(col("Fine amount") == row1["max(Fine amount)"], True))
#).show()



import pyspark.sql.functions as F

cnt_cond = lambda cond: F.sum(F.when(cond, 1).otherwise(0))
citations_new_floatAmount.groupBy('Fine amount').agg(
    cnt_cond(F.col('Fine amount') == row1["max(Fine amount)"]).alias('Fine amount')
).show()


citations_new_floatAmount.select(citations_new_floatAmount['Fine amount'] == row1["max(Fine amount)"]).count()

<<STILL WORKING>>


505.0
1
+-----------+-----------+
|Fine amount|Fine amount|
+-----------+-----------+
|       58.0|          0|
|       30.0|          0|
|      105.0|          0|
|      353.0|          0|
|      163.0|          0|
|      143.0|          0|
|      113.0|          0|
|      255.0|          0|
|       null|          0|
|      293.0|          0|
|      345.0|          0|
|       55.0|          0|
|       70.0|          0|
|       65.0|          0|
|       10.0|          0|
|      505.0|          6|
|       85.0|          0|
|       53.0|          0|
|       68.0|          0|
|      128.0|          0|
+-----------+-----------+
only showing top 20 rows



9257460

<class 'pandas.core.frame.DataFrame'>
   Year  Inflation Rate
0  2016            4.47
1  2017            5.00
2  2018            5.98
3  2019            4.10
Year               object
Inflation Rate    float64
dtype: object
